<a href="https://www.nvidia.com/dli"> <img src="../images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 평가: 저자 판별
### (NVIDIA NeMo v1.0)

저작권 표시는 일종의 텍스트 분류 문제입니다.  질병 텍스트 분류 문제에서 그랬던 것처럼 텍스트를 _주제_별로 분류하는 대신 텍스트를 _저자_별로 분류하는 것이 목적입니다.  

이와 같은 문제를 해결하려고 시도하는 데 있어서 본질적인 가정은 해당 저자들의 *문체에 몇 가지 차이*가 있으며 *모델이 이러한 차이를 알아차릴 수 있다는 것입니다*.  이것이 BERT 등에도 해당될까요?  언어 모델이 문체를 "이해"할 수 있을까요? 

### 목차
[문제](#문제)<br>
[채점](#채점)<br>
[1단계: 데이터 준비](#1단계:-데이터-준비)<br>
[2단계: 모델 Configuration 준비](#2단계:-모델-Configuration-준비)<br>
[3단계: Trainer Configuration 준비](#3단계:-Trainer-Configuration-준비)<br>
[4단계: 트레이닝](#4단계:-트레이닝)<br>
[5단계: 추론](#5단계:-추론)<br>
[6단계: 평가 제출](#6단계:-평가-제출)

# 문제
### 연방주의자 논집 - 역사 미스터리!

[연방주의자 논집](https://en.wikipedia.org/wiki/The_Federalist_Papers)은 1787년과 1788년 사이에 [Alexander Hamilton](https://en.wikipedia.org/wiki/Alexander_Hamilton), [James Madison](https://en.wikipedia.org/wiki/James_Madison), [John Jay](https://en.wikipedia.org/wiki/John_Jay)가 작성한 에세이집입니다.  처음에 'Publius'라는 필명으로 발행되었으며 저자들의 의도는 당시 새로운 미국 헌법의 비준을 독려하는 것이었습니다.  후년에 가서 85개 논문의 각 저자가 확인된 목록이 알려졌습니다.  그럼에도 불구하고 이러한 논문의 하위 세트는 저자가 여전히 알려져 있지 않습니다.  연방주의자 논집 저자 판별 문제는 과거에 진행된 대부분의 통계 NLP 연구의 주제였습니다.   이제 자체 BERT 기반 프로젝트 모델을 사용하여 이 문제를 해결해 보겠습니다.
<img style="float: right;" src="images/HandM.png" width=400>
                                                                                                           
구체적으로 말해서, 문제는 논쟁이 되고 있는 각 논문의 저자가 Alexander Hamilton인지, 아니면 James Madison인지 알아내는 것입니다.  이 연습에서는 각 논문의 저자가 한 명이며(즉, *100%* 확실하지는 않지만 공저가 이루어지지 않았음) 각 저자는 확인된 모든 논문 전체에서 보여진 잘 정의된 문체를 지지고 있다고 가정할 수 있습니다. 

### 프로젝트
이 프로젝트를 위해 레이블이 지정된 `train.tsv` 및 `dev.tsv` 데이터세트가 제공됩니다.  논쟁이 되고 있는 각 논문에 하나씩, 총 10개의 테스트 세트가 있습니다.  모든 데이터세트는 `data/federalist_papers_HM` 디렉토리에 포함되어 있습니다.  

각 "문장"은 사실상 약 256개의 단어로 구성된 문장의 그룹입니다.  레이블은 HAMILTON의 경우 '0', MADISON의 경우 '1'입니다.  예제 파일에는 Madison의 논문보다 Hamilton의 논문이 더 많습니다.  대략적으로 동일한 분포의 레이블 2개를 트레이닝 세트로 사용해 검증 세트를 만들었습니다.

여러분이 해야 할 작업은 랩 2에서 했던 것처럼 NeMo를 사용해 뉴럴 네트워크를 구축하는 것입니다.  모델을 트레이닝하고 테스트하게 됩니다.  그런 다음 제공된 대조 코드를 사용해 모델이 이 "역사 미스터리"에 제공하는 답변을 확인해 보십시오! 

이 과정에서 코드 스니펫을 저장한 다음 오토그레이더(autograder)를 사용해 테스트합니다.  이 부분은 노트북 끝의 제출 지침을 따르십시오.

---
# 채점
최종 결과가 아닌 프로젝트에 맞게 모델을 설정하고 트레이닝하는 능력을 평가받게 됩니다.  이 코딩 평가는 70점 만점이며, 다음과 같이 나뉘어집니다.

### 지시문

| 단계                                | 평가                                                    | FIXMEs?  | 점수 |
|--------------------------------------|-----------------------------------------------------------|----------|--------|
| 1. 프로젝트 준비               | 데이터 형식 수정 (정확한 형식)                          |  2       | 10     |
| 2. 모델 Configuration 준비   | 재정의를 위한 모델 파라미터 설정                         |  3       | 15     |
| 3. Trainer Configuration 준비 | 재정의를 위한 trainer parameters 설정                       |  3       | 15     |
| 4. 트레이닝                            | 트레이너 실행(트레이닝 로그가 트레이닝이 올바름을 나타냄) |  4       | 20      |
| 5. 추론                             | 추론 실행(결과가 유효한 프로젝트임을 나타냄)          |  0       | 10     |

이 시점에서 여러분은 도움을 전혀 받지 않고도 프로젝트를 매우 능숙하게 구축할 수 있지만 특정한 변수 이름을 포함해 몇 가지 리소스가 주어집니다.  이는 오토그레이더(autograder)를 위한 것이므로 평가에 이러한 구조를 사용하십시오.  또한, 이번 평가는 커맨트 라인 메소드를 테스트하고 `text_classification_with_bert.py` 스크립트와 커맨드라인 트레이닝 메소드의 사용을 가정하고 있습니다. 모델 이름, 시퀀스 길이, 배치 크기, 학습률, 에포크 수 등과 같은 매개변수를 자유롭게 변경하여 모델을 적합하게 개선할 수 있습니다. 

신뢰할 수 있는 모델을 구축했다는 확신이 들면 노트북 끝의 제출 지침을 따르십시오.

### 리소스 및 힌트
* **예제 코드:**<br>
왼쪽의 파일 탐색기에서 `lab2_reference_notebooks` 디렉토리를 찾습니다.  여기에는 예제로 사용할 NER 및 텍스트 분류를 위한 랩 2의 솔루션 노트북이 포함되어 있습니다.
* **언어 모델 (PRETRAINED_MODEL_NAME):**<br>
문체를 더 잘 구분하려면 다양한 언어 모델을 시도하는 것이 도움이 된다는 사실을 알게 될 수 있습니다.  구체적으로 말해서, 대문자 사용이 중요할 수 있으며 이는 "대소문자를 구분하는" 모델을 시도해 볼 수 있음을 의미합니다.
* **최대 시퀀스 길이 (MAX_SEQ_LEN):**<br>
MAX_SEQ_LENGTH에 사용할 수 있는 값은 64, 128 또는 256입니다.  더 큰 모델(BERT-large, Megatron)에는 메모리 부족 오류를 피하기 위해 더 작은 MAX_SEQ_LENGTH가 필요할 수 있습니다.
* **클래스 수 (NUM_CLASSES):**<br>
연방주의자 논집의 경우 우리는 Hamilton과 Madison에만 관심을 가졌습니다.  John Jay가 작성한 논문은 데이터세트에서 제외되었습니다.
* **배치 크기 (BATCH_SIZE):**<br>
배치 크기가 크면 더 빨리 트레이닝할 수 있지만 대규모 언어 모델은 가용 메모리를 빠르게 소모합니다.
* **메모리 사용량:**<br>
일부 모델은 규모가 매우 큽니다.   트레이닝 도중 "런타임 오류: CUDA 메모리 부족"이라는 메시지가 수신되면 배치 크기 또는 시퀀스 길이를 줄이거나 더 작은 언어 모델을 선택하고 커널을 다시 시작한 다음 노트북의 처음부터 다시 시도하십시오.
* **정확도 및 손실:**<br>
이 프로젝트에서는 분명 95% 이상의 정확도를 달성할 수 있습니다.  모델을 트레이닝할 때 정확도 변화 외에도 손실 값에 주의를 기울이십시오.  손실 값을 매우 작게 줄여서 최상의 결과를 얻을 수 있습니다.
* **에포크 수(NUM_EPOCHS):**<br>
모델을 위해 더 많은 에포크를 실행해야 할 수도 있습니다(또는 아닐 수도 있음!).  

---
# 1단계: 데이터 준비

In [1]:
# Import useful utilities for grading
import os
import json
import glob
from omegaconf import OmegaConf

def get_latest_model():  
    nemo_model_paths = glob.glob('nemo_experiments/TextClassification/*/checkpoints/*.nemo')
    # Sort newest first
    nemo_model_paths.sort(reverse=True)
    return nemo_model_paths[0]

데이터는 데이터 디렉토리에 위치하고 있습니다. 다음 셀에서 표시하는 목록을 참고하십시오.

In [2]:
DATA_DIR = '/dli/task/data/federalist_papers_HM'
!ls $DATA_DIR

dev.tsv   test49.tsv  test51.tsv  test53.tsv  test55.tsv  test57.tsv  train.tsv
test.tsv  test50.tsv  test52.tsv  test54.tsv  test56.tsv  test62.tsv


## 데이터 형식 (평가됨)
데이터는 NeMo 텍스트 분류에 올바른 형식이 아닙니다. 데이터를 수정하고 DATA_DIR 에 새로운 데이터 세트를 `train_nemo_format.tsv`및 `dev_nemo_format.tsv`로 저장합니다.  테스트 파일을 사용할 필요가 없습니다.  

<i><strong style="color:green;">#FIXME</strong></i> 라인을 완성하고 저장된 셀을 실행하세요.

In [3]:
# Correct the format for train.tsv and dev.tsv
#   and save the updates in train_nemo_format.tsv and dev_nemo_format.tsv

#FIXME train.tsv format
#FIXME dev.tsv format

!sed 1d $DATA_DIR/train.tsv > $DATA_DIR/train_nemo_format.tsv
!sed 1d $DATA_DIR/dev.tsv > $DATA_DIR/dev_nemo_format.tsv

In [4]:
# check your work
print("*****\ntrain_nemo_format.tsv sample\n*****")
!head -n 3 $DATA_DIR/train_nemo_format.tsv
print("\n\n*****\ndev_nemo_format.tsv sample\n*****")
!head -n 3 $DATA_DIR/dev_nemo_format.tsv

*****
train_nemo_format.tsv sample
*****
Concerning Dangers from Dissensions Between the States For the Independent Journal .To the People of the State of New York : THE three last numbers of this paper have been dedicated to an enumeration of the dangers to which we should be exposed , in a state of disunion , from the arms and arts of foreign nations .I shall now proceed to delineate dangers of a different and , perhaps , still more alarming kind -- those which will in all probability flow from dissensions between the States themselves , and from domestic factions and convulsions .These have been already in some instances slightly anticipated ; but they deserve a more particular and more full investigation .A man must be far gone in Utopian speculations who can seriously doubt that , if these States should either be wholly disunited , or only united in partial confederacies , the subdivisions into which they might be thrown would have frequent and violent contests with each other .To

In [5]:
# Run to save for assessment- DO NOT CHANGE
import os.path
DATA_DIR = '/dli/task/data/federalist_papers_HM'
step1 = []
try:
    with open(os.path.join(DATA_DIR,'train_nemo_format.tsv')) as f:
        content = f.readlines()
        step1 += content[:2]
    with open(os.path.join(DATA_DIR,'dev_nemo_format.tsv')) as f:
        content = f.readlines()
        step1 += content[:2]
except:
    pass
                
with open("my_assessment/step1.json", "w") as outfile: 
    json.dump(step1, outfile) 

---
# 2단계: 모델 Configuration 준비
기본 모델 configuration 과 가용한 언어 모델을 검토합니다. 

In [6]:
# Take a look at the default model portion of the config file
CONFIG_DIR = "/dli/task/nemo/examples/nlp/text_classification/conf"
CONFIG_FILE = "text_classification_config.yaml"

config = OmegaConf.load(CONFIG_DIR + "/" + CONFIG_FILE)
print(OmegaConf.to_yaml(config.model))

nemo_path: text_classification_model.nemo
tokenizer:
  tokenizer_name: ${model.language_model.pretrained_model_name}
  vocab_file: null
  tokenizer_model: null
  special_tokens: null
language_model:
  pretrained_model_name: bert-base-uncased
  lm_checkpoint: null
  config_file: null
  config: null
classifier_head:
  num_output_layers: 2
  fc_dropout: 0.1
class_labels:
  class_labels_file: null
dataset:
  num_classes: ???
  do_lower_case: false
  max_seq_length: 256
  class_balancing: null
  use_cache: false
train_ds:
  file_path: null
  batch_size: 64
  shuffle: true
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
validation_ds:
  file_path: null
  batch_size: 64
  shuffle: false
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
test_ds:
  file_path: null
  batch_size: 64
  shuffle: false
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
optim:
  name: adam
  lr: 2.0e-05
  betas:
  - 0.9
  - 0.999
  weight_decay:

In [7]:
# See what BERT-like language models are available
from nemo.collections import nlp as nemo_nlp
nemo_nlp.modules.get_pretrained_lm_models_list()

['megatron-bert-345m-uncased',
 'megatron-bert-345m-cased',
 'megatron-bert-uncased',
 'megatron-bert-cased',
 'biomegatron-bert-345m-uncased',
 'biomegatron-bert-345m-cased',
 'bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'bert-base-german-dbmdz-cased',
 'bert-base-german-dbmdz-uncased',
 'cl-tohoku/bert-base-japanese',
 'cl-tohoku/bert-base-japanese-whole-word-masking',
 'cl-tohoku/bert-base-japanese-char',
 'cl-tohoku/bert-base-japanese-char-whole-word-masking',
 'TurkuNLP/bert-base-finnish-cased-v1',
 'TurkuNLP/bert-base-finnish-uncased-v1',
 'wietsedv/bert-base-dutch-cased',
 'distilbert-base-uncased

## 파라미터 설정하기 (평가됨)
<i><strong style="color:green;">#FIXME</strong></i> 라인을 완성하고 저장된 셀을 실행하세요.

In [22]:
# set the values
NUM_CLASSES = 2 #FIXME 
MAX_SEQ_LENGTH = 256 #FIXME 
BATCH_SIZE = 16 #FIXME 
PATH_TO_TRAIN_FILE = "/dli/task/data/federalist_papers_HM/train_nemo_format.tsv"
PATH_TO_VAL_FILE = "/dli/task/data/federalist_papers_HM/dev_nemo_format.tsv"
PRETRAINED_MODEL_NAME = 'bert-base-cased' # change as desired
# PRETRAINED_MODEL_NAME = 'bert-base-uncased' # change as desired
LR = 1e-4 # change as desired

In [23]:
# Run to save for assessment- DO NOT CHANGE
with open("my_assessment/step2.json", "w") as outfile: 
    json.dump([MAX_SEQ_LENGTH, NUM_CLASSES, BATCH_SIZE], outfile) 

---
# 3단계: Trainer Configuration 준비
기본 trainer 와 exp_manager configuration를 검토합니다.

In [24]:
print(OmegaConf.to_yaml(config.trainer))
print(OmegaConf.to_yaml(config.exp_manager))

gpus: 1
num_nodes: 1
max_epochs: 100
max_steps: null
accumulate_grad_batches: 1
gradient_clip_val: 0.0
amp_level: O0
precision: 32
accelerator: ddp
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null
num_sanity_val_steps: 0
checkpoint_callback: false
logger: false

exp_dir: null
name: TextClassification
create_tensorboard_logger: true
create_checkpoint_callback: true



## 파라미터 지정 (평가됨)
자동 혼합 정밀도를 FP16 정밀도로 레벨 1로 설정합니다. MAX_EPOCHS to를 적절한 수준 (약 5~20)으로 설정합니다.  <br><i><strong style="color:green;">#FIXME</strong></i> 라인을 완성하고 저장된 셀을 실행하세요.

In [25]:
# set the values
# MAX_EPOCHS = #FIXME 
# AMP_LEVEL = #FIXME 
# PRECISION = #FIXME 

MAX_EPOCHS = 10
AMP_LEVEL = 'O1'
PRECISION = 16

In [26]:
# Run to save for assessment - DO NOT CHANGE
with open("my_assessment/step3.json", "w") as outfile: 
    json.dump([MAX_EPOCHS, AMP_LEVEL, PRECISION], outfile) 

---
# 4단계: 트레이닝

### Trainer 실행하기 (평가됨)
<i><strong style="color:green;">#FIXME</strong></i>를 완성하시고 다음 셀에서 트레이닝 및 검증 배치 사이즈, amp 레벨과 정밀도를 확인하세요.  그 다음 트레이닝을 진행 후 저장된 셀을 실행하세요!

In [27]:
%%time
# Run the training script, overriding the config values in the command line
TC_DIR = "/dli/task/nemo/examples/nlp/text_classification"


!python $TC_DIR/text_classification_with_bert.py \
        model.dataset.num_classes=$NUM_CLASSES \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.file_path=$PATH_TO_TRAIN_FILE \
        model.validation_ds.file_path=$PATH_TO_VAL_FILE \
        model.infer_samples=[] \
        trainer.max_epochs=$MAX_EPOCHS \
        model.language_model.pretrained_model_name=$PRETRAINED_MODEL_NAME \
        model.train_ds.batch_size=$BATCH_SIZE \
        model.validation_ds.batch_size=$BATCH_SIZE \
        trainer.amp_level=$AMP_LEVEL \
        trainer.precision=$PRECISION \
        model.optim.lr=$LR

[NeMo W 2022-08-23 07:49:03 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/omegaconf/basecontainer.py:225: UserWarning: cfg.pretty() is deprecated and will be removed in a future version.
    Use OmegaConf.to_yaml(cfg)
    
      warnings.warn(
    
[NeMo I 2022-08-23 07:49:03 text_classification_with_bert:110] 
    Config Params:
    trainer:
      gpus: 1
      num_nodes: 1
      max_epochs: 10
      max_steps: null
      accumulate_grad_batches: 1
      gradient_clip_val: 0.0
      amp_level: O1
      precision: 16
      accelerator: ddp
      log_every_n_steps: 1
      val_check_interval: 1.0
      resume_from_checkpoint: null
      num_sanity_val_steps: 0
      checkpoint_callback: false
      logger: false
    model:
      nemo_path: text_classification_model.nemo
      tokenizer:
        tokenizer_name: ${model.language_model.pretrained_model_name}
        vocab_file: null
        tokenizer_model: null
        special_tokens: null
      language_model:
        pretrain

In [28]:
# Run to save for assessment- DO NOT CHANGE
cmd_log = os.path.join(os.path.dirname(os.path.dirname(get_latest_model())),'cmd-args.log')
lightning_logs = os.path.join(os.path.dirname(os.path.dirname(get_latest_model())),'lightning_logs.txt')

with open(cmd_log, "r") as f:
    cmd = f.read()
    cmd_list = cmd.split()
with open("my_assessment/step4.json", "w") as outfile: 
    json.dump(cmd_list, outfile) 
    
with open(lightning_logs, "r") as f:
    log = f.readlines()
with open("my_assessment/step4_lightning.json", "w") as outfile:
    json.dump(log, outfile)

---
# 5단계: 추론

### 추론 실행하기 (평가됨)
추론 블록을 실행하고 결과를 확인하고 저장합니다. (노트: 여기는 추가로 고칠 부분이 없습니다.)

In [29]:
# Run inference for assessment -  - DO NOT CHANGE
from nemo.collections import nlp as nemo_nlp

# Instantiate the model by restoring from the latest .nemo checkpoint
model = nemo_nlp.models.TextClassificationModel.restore_from(get_latest_model())

# Find the latest model path
DATA_DIR = '/dli/task/data/federalist_papers_HM'

test_files = [
    'test49.tsv',
    'test50.tsv',
    'test51.tsv',
    'test52.tsv',
    'test53.tsv',
    'test54.tsv', 
    'test55.tsv',
    'test56.tsv',
    'test57.tsv',
    'test62.tsv',
]
results = []
for test_file in test_files:
    # get as list and remove header row
    filepath = os.path.join(DATA_DIR, test_file)
    with open(filepath, "r") as f:
        lines = f.readlines()
    del lines[0]
    
    results.append(model.classifytext(lines, batch_size = 1, max_seq_length = 256))
print(results)

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
[NeMo W 2022-08-23 07:51:59 modelPT:137] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    file_path: /dli/task/data/federalist_papers_HM/train_nemo_format.tsv
    batch_size: 16
    shuffle: true
    num_samples: -1
    num_workers: 3
    drop_last: false
    pin_memory: false
    
[NeMo W 2022-08-23 07:51:59 modelPT:144] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    file_path: /dli/task/data/federalist_papers_HM/dev_nemo_format.tsv
    batch_size: 16
    shuffle: false
    num_samples: -1
    num_workers: 3
    drop_last: false
    pin_memory: false
    
[NeMo W 2022-08-23 0

[NeMo I 2022-08-23 07:52:04 modelPT:434] Model TextClassificationModel was successfully restored from nemo_experiments/TextClassification/2022-08-23_07-49-03/checkpoints/TextClassification.nemo.


[NeMo W 2022-08-23 07:52:04 text_classification_dataset:250] Found 7 out of 7 sentences with more than 256 subtokens. Truncated long sentences from the end.
[NeMo W 2022-08-23 07:52:05 text_classification_dataset:250] Found 4 out of 4 sentences with more than 256 subtokens. Truncated long sentences from the end.
[NeMo W 2022-08-23 07:52:06 text_classification_dataset:250] Found 8 out of 8 sentences with more than 256 subtokens. Truncated long sentences from the end.
[NeMo W 2022-08-23 07:52:06 text_classification_dataset:250] Found 7 out of 7 sentences with more than 256 subtokens. Truncated long sentences from the end.
[NeMo W 2022-08-23 07:52:06 text_classification_dataset:250] Found 9 out of 9 sentences with more than 256 subtokens. Truncated long sentences from the end.
[NeMo W 2022-08-23 07:52:07 text_classification_dataset:250] Found 8 out of 8 sentences with more than 256 subtokens. Truncated long sentences from the end.
[NeMo W 2022-08-23 07:52:07 text_classification_dataset:25

[[0, 1, 1, 1, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0, 1, 1, 1, 0], [0, 1, 1, 1, 1, 0, 0], [0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 1, 1, 1, 1], [1, 1, 0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 1, 1], [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]]


In [30]:
# Run to save for assessment- DO NOT CHANGE
author = []
for result in results:
    avg_result = sum(result) / len(result)
    if avg_result < 0.5:
        author.append("HAMILTON")
        print("HAMILTON")
    else:
        author.append("MADISON")
        print("MADISON")
        
with open("my_assessment/step5.json", "w") as outfile: 
    json.dump(author, outfile) 

MADISON
HAMILTON
MADISON
MADISON
HAMILTON
MADISON
MADISON
HAMILTON
HAMILTON
HAMILTON


# 6단계: 평가 제출
결과가 어땠습니까?  [위키디피아 문서](https://en.wikipedia.org/wiki/The_Federalist_Papers)에 따르면 과거의 전문가들은 Madison이 논쟁이 되고 있는 모든 논문의 저자라고 믿었지만 최근 분석에서는 일부 공저의 가능성도 제시됩니다.  갖고 있는 도구를 사용해 "모두 MADISON"이라는 답을 얻을 수도 있습니다.  원한다면 계속 시도해 봐도 되지만 **특정 결과는 평가에 합격하는 데 필요하지 *않습니다***.

코드를 올바르게 작성했고 트레이닝 및 추론이 올바르게 작동하고 있다고 확신한다면 다음과 같이 오토그레이더(autograder)에 프로젝트를 제출할 수 있습니다.

1. GPU 시작 페이지로 돌아가서 체크 표시를 클릭해 평가를 실행합니다.

<img src="../images/assessment_checkmark.png" width=600>

2. 잘 하셨습니다!  합격했으면 합격을 알리는 팝업 창이 표시되고 진도에 점수가 추가됩니다.  그렇지 않은 경우 팝업 창에 피드백이 표시됩니다. 

<img src="../images/assessment_pass_popup.png" width=600>

과정 진도 탭에서 항상 평가 진행 상황을 확인할 수 있습니다.  코딩 평가의 부분 값은 여기에 표시되지 않으며, 0점 또는 70점으로 표시됩니다.  동일한 과정 페이지의 트랜스포머 및 배포에서 질문을 완료하면 최종 인증서를 받을 수 있는 자격이 부여됩니다!

<img src="../images/progress.png" width=600>

<a href="https://www.nvidia.com/dli"> <img src="../images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>